In [7]:
import torch
from transformers import BertTokenizer,BertForSequenceClassification,AdamW,get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [14]:
if torch.cuda.is_available():
    print('GPU')
    device=torch.device("cuda")
else:
    print('CPU')
    device = torch.device("cpu")

CPU


In [15]:
train=pd.read_csv("../data/train.csv")
valid=pd.read_csv("../data/valid.csv")
test=pd.read_csv("../data/test.csv")

In [16]:
#Initialize BertTokenzier
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [18]:
tokenized_train=tokenizer.batch_encode_plus(train.iloc[:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')
labels_train=torch.tensor(train.label.values[:])

tokenized_val=tokenizer.batch_encode_plus(valid.iloc[-1000:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')
labels_val=torch.tensor(valid.label.values[-1000:])

tokenized_test=tokenizer.batch_encode_plus(test.iloc[:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')

C:\repos\NLP_H22_ATP\venv\lib\site-packages\transformers\tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overfl

In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

batch_size=32

train_data=TensorDataset(tokenized_train['input_ids'],tokenized_train['attention_mask'],tokenized_train['token_type_ids'],labels_train)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data=TensorDataset(tokenized_val['input_ids'],tokenized_val['attention_mask'],tokenized_val['token_type_ids'],labels_val)
val_sampler=RandomSampler(val_data)
val_dataloader=DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

test_data=TensorDataset(tokenized_test['input_ids'],tokenized_test['attention_mask'],tokenized_test['token_type_ids'])
# test_sampler=RandomSampler(test_data)
test_dataloader=DataLoader(test_data, batch_size=batch_size)

In [21]:
#Bert Model transformer with a single sequence classification layer on top
model=BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=3,output_attentions=False,output_hidden_states=False)
model()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

ValueError: You have to specify either input_ids or inputs_embeds

In [ ]:
#Set learning rate
optimizer=AdamW(model.parameters(),lr=2e-5)
epochs=8

#Training steps is no_of_batches*no_of_epochs
total_steps=len(train_dataloader)*epochs

#Learning rate scheduler
scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps = total_steps)

In [ ]:
# Function to calculate the accuracy of predictions
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import random

seed_val = 12

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):

    #Put model into training mode
    model.train()

    total_loss=0

    for step, batch in enumerate(train_dataloader):

        #Unpack the training batch
        b_input_ids = batch[0].to(device)
        b_attention_mask=batch[1].to(device)
        b_token_type = batch[2].to(device)
        b_labels = batch[3].to(device)

        #Clear previously calculated gradients before performing a backward pass
        #model.zero_grad()        //Not sure if useful or not

        #Perform a forward pass and get the loss
        outputs=model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask,labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()

        #Perform backward pass to calculate gradients
        loss.backward()

        #Clip the norm of the gradients to 1.0.
        #Used to prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #Update weights
        optimizer.step()

        #Update learning rate
        scheduler.step()

    #Avg loss over training data for an epoch
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    #### Validation

    #Evaluation mode
    model.eval()

    #Tracking variables
    val_accuracy=0
    nb_val_steps=0

    #Evaluate data for one epoch
    for batch in val_dataloader:

        #Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        #Unpack the inputs from dataloader
        b_input_ids,b_attention_mask, b_token_type, b_labels = batch

        #Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            outputs = model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask)

        #Get the "logits" output by the model. The "logits" are output values prior to applying an activation function like the softmax.
        logits = outputs[0]

        #Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        #Calculate the accuracy for this batch of test sentences.
        tmp_val_accuracy = flat_accuracy(logits, label_ids)

        #Accumulate the total accuracy.
        val_accuracy += tmp_val_accuracy

        #Track the number of batches
        nb_val_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(val_accuracy/nb_val_steps))

In [ ]:
final_output = []
for batch in test_dataloader:
  #Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  #Unpack the inputs from dataloader
  b_input_ids,b_attention_mask, b_token_type= batch

  #Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
      outputs = model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask)

  #Get the "logits" output by the model. The "logits" are output values prior to applying an activation function like the softmax.
  logits = outputs[0]
  final_output.extend(np.argmax(logits.detach().cpu().numpy(), axis=1).flatten())

In [ ]:
output = pd.DataFrame({'id': train.id,
                       'prediction': final_output})
output.to_csv('submission.csv', index=False)